In [1]:
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import glob

In [2]:
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

In [3]:
data = []
labels = []

In [4]:
image_files = [f for f in glob.glob(r'C:\Users\Paret\Desktop\Coding\CNN practice\Gender-Detection-master\Gender-Detection-master\gender_dataset_face' + "/**/*", recursive = True) if not os.path.isdir(f)]
random.shuffle(image_files)

In [5]:
for img in image_files:
    
    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0], img_dims[1]))
    image = img_to_array(image)
    data.append(image)
    
    label = img.split(os.path.sep)[-2]
    if label == "woman":
        label = 1
    else:
        label = 0
    
    labels.append([label])

In [6]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [7]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

In [8]:
trainY = to_categorical(trainY, num_classes=2) 
testY = to_categorical(testY, num_classes=2)

In [9]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [10]:
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": 
        inputShape = (depth, height, width)
        chanDim = 1
   
    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

In [11]:
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

In [12]:
opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [13]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
28/28 [==============================] - 16s 570ms/step - loss: 0.8102 - accuracy: 0.7007 - val_loss: 0.6632 - val_accuracy: 0.5628
Epoch 2/100
28/28 [==============================] - 16s 564ms/step - loss: 0.4332 - accuracy: 0.8265 - val_loss: 0.6916 - val_accuracy: 0.5649
Epoch 3/100
28/28 [==============================] - 16s 583ms/step - loss: 0.3943 - accuracy: 0.8495 - val_loss: 0.6980 - val_accuracy: 0.5671
Epoch 4/100
28/28 [==============================] - 16s 585ms/step - loss: 0.3763 - accuracy: 0.8546 - val_loss: 0.6432 - val_accuracy: 0.5649
Epoch 5/100
28/28 [==============================] - 16s 588ms/step - loss: 0.3408 - accuracy: 0.8613 - val_loss: 0.9377 - val_accuracy: 0.5584
Epoch 6/100
28/28 [==============================] - 16s 589ms/step - loss: 0.3027 - accuracy: 0.8798 - val_loss: 0.9888 - val_accuracy: 0.5584
Epoch 7/100
28/28 [==============================] - 16s 585

In [14]:
model.save('gender_detection')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: gender_detection\assets
